In [ ]:
# TASK 3 – PS ↔ PL GPIO CALCULATOR (SINGLE-PAGE JUPYTER NOTEBOOK)
#
# hw:
#   - ZYNQ PS + PL
#   - EMIO GPIO, 52-bit wide
#   - RTL: task3.v
#
# communication model:
#   - NO AXI
#   - NO MMIO
#   - Pure GPIO read/write
#
# GPIO bit layout (LSB → MSB):
#   [23:0]   : inputs passthrough (in0-in1-in2 packed)
#   [47:24]  : output
#   [48]     : done
#   [51:49]  : opcode
#
# author: Alp Bolukbasi


from pynq import Overlay, GPIO
from dataclasses import dataclass
import time
from typing import Tuple

# constants and bitmasks
GPIO_WIDTH   = 52

INPUT_LSB    = 0
RESULT_LSB   = 24
DONE_BIT     = 48
OPCODE_LSB   = 49

MASK_24      = (1 << 24) - 1
MASK_3       = (1 << 3)  - 1
MASK_1       = 1

# load overlay
# bitstream and .hwh must have the same name
overlay = Overlay("task3.bit")
overlay.download()

# GPIO binding
# single wide EMIO GPIO, bidirectional
gpio = GPIO(GPIO.get_gpio_pin(0), "inout")


# structured result container, as dataclass
@dataclass
class CalcResult:
    opcode: int
    input_raw: int
    result: int
    done: bool

# low level bit packaging/unpacking functions
def pack_inputs(opcode: int, in0: int, in1: int, in2: int) -> int:
    """
    pack opcode + inputs into 52-bit GPIO word
    inputs are packed exactly as in RTL expectations.
    """
    inputs_24 = ((in2 & 0xFF) << 16) | ((in1 & 0xFF) << 8) | (in0 & 0xFF)
    word = 0
    word |= (inputs_24 & MASK_24) << INPUT_LSB
    word |= (opcode & MASK_3)   << OPCODE_LSB
    return word


def unpack_outputs(word: int) -> Tuple[int, bool]:
    """
    extract result and done flag from GPIO read
    """
    result = (word >> RESULT_LSB) & MASK_24
    done   = (word >> DONE_BIT) & MASK_1
    return result, bool(done)

# highlevel calculator interface
class PLCalculator:
    """
    clean PS-side abstraction for task3.v
    """

    def __init__(self, gpio):
        self.gpio = gpio

    def execute(self, opcode: int, in0: int = 0, in1: int = 0, in2: int = 0) -> CalcResult:
        """
        sends inputs to PL, waits for 'done', returns result
        """
        word_out = pack_inputs(opcode, in0, in1, in2)
        self.gpio.write(word_out)

        # polling loop (done is always 1 in the RTL, but this is future-safe)
        while True:
            word_in = self.gpio.read()
            result, done = unpack_outputs(word_in)
            if done:
                return CalcResult(
                    opcode=opcode,
                    input_raw=word_out,
                    result=result,
                    done=done
                )
            time.sleep(0.001)

# opcode references from task3.v
OPCODE_MAP = {
    0: "ADD        : in1 + in2",
    1: "SUB        : in2 - in1",
    2: "MULT       : in1 * in2",
    3: "SHIFT-R    : in2 >> in1",
    4: "SQUARE     : in1^2",
    5: "CUBE       : in1^3",
    6: "TRIPLE ADD : in0 + in1 + in2",
    7: "POLYNOMIAL : 5x² + 8x - 4y² + 3y + 6z² - 2z + 13"
}

# instantiation
calc = PLCalculator(gpio)


# functional tests
tests = [
    (0, 0, 5, 7),        # ADD
    (1, 0, 3, 10),       # SUB
    (2, 0, 4, 6),        # MULT
    (3, 0, 1, 8),        # SHIFT
    (4, 0, 9, 0),        # SQUARE
    (5, 0, 4, 0),        # CUBE
    (6, 1, 2, 3),        # TRIPLE ADD
    (7, 0x3, 0x2, 0x1),  # POLYNOMIAL (x=1,y=2,z=3 via nibble usage)
]

print("\ntask 3 results:\n")

for opcode, in0, in1, in2 in tests:
    res = calc.execute(opcode, in0, in1, in2)
    print(f"opcode {opcode}: {OPCODE_MAP[opcode]}")
    print(f"  inputs  -> in0={in0}, in1={in1}, in2={in2}")
    print(f"  result  -> {res.result}")
    print(f"  done    -> {res.done}")
    print("-" * 55)

print("\nall operations executed via GPIO-only PS/PL interface\n")
